# Wine List Dashboard

In [48]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from geocodes import add_coordinates

import warnings
warnings.filterwarnings('ignore')

In [55]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Set the mapbox access token
px.set_mapbox_access_token(map_box_api)

## Import Required Data

In [50]:
# Loading Wine Data
file_path = Path('./Data/wine_data.csv')
wine_data = pd.read_csv(file_path, encoding='UTF-8')

# Loading Census Data
file_path = Path('./Data/census_data.csv')
census_data = pd.read_csv(file_path, encoding='UTF-8')

In [51]:
# Census data analysis func
def census_data_output():
    top_income_df = census_data.loc[(census_data['Income']>=60000) & (census_data['Income']<=80000)]
    df = top_income_df.groupby(['State', 'County']).sum()
    df['Rank'] = df[['Marital_Pop', 'Age_Pop', 'Education_Pop']].sum(axis=1)
    df = df.sort_values('Rank', ascending=False).reset_index()
    df = add_coordinates(df)
    
    return df

census_data = census_data_output()

## Panel Visualizations

In [72]:
# All pannel Visualizations functions

def wine_con_age_groups():
    ''' Wine consumption based on Age Groups '''
    
    df = wine_data.groupby('Age Group').count()['CustId']
    fig = df.hvplot.bar(x='Age Group', xlabel='Age Group', ylabel='Number of Consumers', title='Wine consumption based on Age Groups')
    
    return fig

def wine_sales_income_bracket():
    ''' Total Wine Sales per Income Bracket '''
    
    df = wine_data.groupby('Income Bracket').sum()['MntWines']
    fig = df.hvplot.barh(x='Income Bracket', y='MntWines', xlabel='Income Bracket', ylabel='Total Wine Sales ($)', xformatter='%.0f', title='Total Wine Sales per Income Bracket')
    
    return fig

def wine_con_marital_stats():
    ''' Wine consumption based on Marital Stats '''
    
    df = wine_data.groupby('MaritalStat').count()['CustId'].reset_index()
    df = df.rename(columns={'CustId':'count'})
    fig = px.pie(df, names='MaritalStat', values='count', title='Wine consumption based on Marital Stats')
    fig.update_traces(textposition='inside', textinfo='percent+label')
    
    return fig

def wine_con_marital_age_groups():
    ''' Wine consumption based on Marital Status and Age Group '''
    
    df = wine_data.groupby(['MaritalStat','Age Group']).count()['CustId'].reset_index()
    df = df.rename(columns={'CustId':'Number of consumers'})
    df = df.loc[df['Number of consumers']!= 0] # Excluding 0 records

    fig = px.sunburst(
        df, 
        path=['MaritalStat','Age Group'],
        values='Number of consumers',
        color='Number of consumers',
        title='Wine consumption based on Marital Status and Age Group')
    
    return fig

def wine_sales_marital_education():
    ''' Total Sales based on Education and Marital Stat '''
    
    df = wine_data.groupby(['Education', 'MaritalStat']).sum()['MntWines']
    fig = df.hvplot.bar(xlabel='Education and Marital Stat', ylabel='Total Sales ($)', title='Total Sales based on Education and Marital Stat', rot=90, yformatter='%.0f')
    
    return fig

def channel_store_web():
    ''' Customer Purchasing channels and their effectiveness based on Store and Web '''
    
    df = wine_data[['MntWines','NumWebPurchases','NumDealsPurchases','NumCatalogPurchases','NumStorePurchases']]
    web_store_total_purchases = df['NumWebPurchases'] + df['NumStorePurchases']

    df['WebSales'] = df['MntWines'] * df['NumWebPurchases'] / (web_store_total_purchases)
    df['StoreSales'] = df['MntWines'] * df['NumStorePurchases'] / (web_store_total_purchases)

    df_webstore = pd.DataFrame({
        'Sales': [df['WebSales'].sum().round(), 
                  df['StoreSales'].sum().round()]
    })

    fig = px.scatter(df_webstore, 
                     x=['Total Web Purchases','Total Store Purchases'],
                     y='Sales',
                     color=['Total Web Purchases','Total Store Purchases'], 
                     labels={'x':'Purchasing Chanel', 'y':'Sales ($)'},
                     size='Sales',
                     title = 'Channel Performance - Store vs Website',
                     size_max=100)

    return fig

def channel_deal_catalog():
    ''' Customer Purchasing channels and their effectiveness based on Deals and Catalog '''
    
    df = wine_data[['MntWines','NumWebPurchases','NumDealsPurchases','NumCatalogPurchases','NumStorePurchases']]
    deal_catalog_total_purchases = df['NumDealsPurchases'] + df['NumCatalogPurchases']
    df['DealsSales'] = df['MntWines'] * df['NumDealsPurchases'] / (deal_catalog_total_purchases)
    df['CatalogSales'] = df['MntWines'] * df['NumCatalogPurchases'] / (deal_catalog_total_purchases)


    df_dealcat = pd.DataFrame({
        'Sales': [df['DealsSales'].sum().round(), 
                  df['CatalogSales'].sum().round()]
    })

    fig = px.scatter(df_dealcat, 
                     x=['Total Deals Purchases','Total Catalog Purchases'],
                     y='Sales',
                     color=['Total Deals Purchases','Total Catalog Purchases'],
                     labels = {'x':'Purchases Mode', 'y':'Sales ($)'},
                     size='Sales', 
                     title = 'Channel Performance - Deals vs Catalogs', 
                     size_max=100)
    
    return fig

def total_sales_per_campaign():
    ''' Total Sales generated for past Marketing Campaigns '''
    
    # Calculating each Marketing Campaign sales figues based on Customer Acceptance
    campaign_1_total_sales = wine_data.loc[wine_data['AcceptedCmp1'] == 1].sum()['MntWines']
    campaign_2_total_sales = wine_data.loc[wine_data['AcceptedCmp2'] == 1].sum()['MntWines']
    campaign_3_total_sales = wine_data.loc[wine_data['AcceptedCmp3'] == 1].sum()['MntWines']
    campaign_4_total_sales = wine_data.loc[wine_data['AcceptedCmp4'] == 1].sum()['MntWines']
    campaign_5_total_sales = wine_data.loc[wine_data['AcceptedCmp5'] == 1].sum()['MntWines']

    # Create a new DataFrame based on these data
    df = pd.DataFrame({
        'Campaign':['Campaign-1', 'Campaign-2', 'Campaign-3', 'Campaign-4', 'Campaign-5'],
        'Total Sales ($)': [campaign_1_total_sales, 
                            campaign_2_total_sales, 
                            campaign_3_total_sales, 
                            campaign_4_total_sales, 
                            campaign_5_total_sales]
    })

    # Added a Trend line over the bar
    fig = px.line(df, x='Campaign', y='Total Sales ($)', title='Total Sales generated for past Marketing Campaigns')

    # Plotting a bar graph
    fig.add_bar(x=['Campaign-1', 'Campaign-2', 'Campaign-3', 'Campaign-4', 'Campaign-5'], 
                y=[campaign_1_total_sales, 
                   campaign_2_total_sales, 
                   campaign_3_total_sales, 
                   campaign_4_total_sales, 
                   campaign_5_total_sales],
                showlegend=False)
    return fig

def food_basket_com():
    ''' Food Basket Comparison '''
    
    df = wine_data[['MntWines','MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts']]
    fig = df.hvplot.line(xlabel = 'Amount',
                         ylabel= 'Number',
                         title = 'Food Basket Comparison')
    
    return fig

def cust_purchasing_with_kids():
    ''' Customers with kids / Teen purchasing wine '''
    
    df = wine_data[['MntWines','NumKids','NumTeen']]
    fig = px.parallel_coordinates(df,
                                  ['NumKids','NumTeen', 'MntWines'],
                                  color = 'MntWines',
                                  title = 'Customers with kids / Teen purchasing wine')
    
    return fig
    
def top_wine_states():
    ''' Top Wine States in US '''
    
    map_plot = px.scatter_mapbox(
        census_data,
        lat='Lat',
        lon='Lon',
        color='State',
        size='Rank',
        title='Top Wine States in US',
        color_continuous_scale=px.colors.sequential.Rainbow,
        size_max=10,
        zoom=3
    )
    
    return map_plot

def top_wine_counties():
    ''' Top Wine Counties in US '''
    
    map_plot = px.scatter_mapbox(
        census_data,
        lat='Lat',
        lon='Lon',
        color='County',
        size='Rank',
        title='Top Wine Counties in US',
        color_continuous_scale=px.colors.sequential.Rainbow,
        size_max=5,
        zoom=3
    )

    return map_plot

In [84]:
# Create the dashboard
welcome = pn.Column(
    "This dashboard provide appropriate customer insights of Wine List Corporation develop a targeted marketing and sales strategy so as to maximize sales dollars and minimize marketing dollars",
    top_wine_states(),
    top_wine_counties(),
    background='#ffe5e5'
)

sales_customer_segments = pn.Column(
    wine_sales_marital_education(),
    wine_sales_income_bracket(),
)

usage_customer_segments = pn.Column(
    pn.Row(
        wine_con_marital_age_groups(),
        wine_con_marital_stats()
    ),
    wine_con_age_groups()
)

sales_channels = pn.Row(
    channel_store_web(),
    channel_deal_catalog()
)

campaign_performance = pn.Column(
    total_sales_per_campaign()
)

other_insights = pn.Column(
    food_basket_com(),
    cust_purchasing_with_kids()
)

wine_tabs = pn.Tabs(
    ('Welcome', welcome),
    ('Sales Customer Segmentation', sales_customer_segments),
    ('Usage Customer Segmentation', usage_customer_segments),
    ('Sales Channels', sales_channels),
    ('Campaign Performances', campaign_performance),
    ('Other Insights', other_insights)
)

wine_dashboard = pn.Column(
    '## Real Estate Analysis of San Francisco from 2010 to 2016',
    wine_tabs
)


In [85]:
# Serve the# dashboard
wine_tabs.servable()

Tabs
    [0] Column(background='#ffe5e5')
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Column
        [0] HoloViews(Bars)
        [1] HoloViews(Bars)
    [2] Column
        [0] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [1] HoloViews(Bars)
    [3] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)
    [4] Column
        [0] Plotly(Figure)
    [5] Column
        [0] HoloViews(NdOverlay)
        [1] Plotly(Figure)